In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib agg
import pyart
import numpy.ma as ma
import matplotlib.ticker as mticker
from matplotlib import colors as c
from matplotlib.colors import ListedColormap,BoundaryNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.geodesic as cargeo
from shapely.geometry import Polygon
import os
import warnings
warnings.filterwarnings('ignore')


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



C:\Users\Admin\anaconda3\envs\pyart_env\lib\site-packages\wradlib\io\radolan.py:939: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  "add_offset": np.float(0),
C:\Users\Admin\anaconda3\envs\pyart_env\lib\site-packages\wradlib\io\radolan.py:948: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  "add_offset": np.float(0),
C:\Users\Admin\anaconda3

In [2]:
class UF:
    def __init__(self,radar,shape_grid,lat_0, lon_0):
        self.radar = radar
        self.shape_grid = shape_grid
        self.lat_0 = lat_0
        self.lon_0 = lon_0
    def remove_noises(self): #Step 3-A
        mask_noises_CC=ma.masked_less(self.radar.fields['cross_correlation_ratio']['data'],0.85)
        mask_noises_Z=ma.masked_less(self.radar.fields['reflectivity']['data'],0)
        rm_noises_Z_=np.ma.masked_array(self.radar.fields['reflectivity']['data'],mask_noises_CC.mask)
        rm_noises_Z=np.ma.masked_array(rm_noises_Z_,mask_noises_Z.mask)
        rm_noises_ZDR_=np.ma.masked_array(self.radar.fields['differential_reflectivity']['data'],mask_noises_CC.mask)
        rm_noises_ZDR=ma.masked_array(rm_noises_ZDR_,mask_noises_Z.mask)
        self.radar.add_field_like('reflectivity','Z_removed_noises',rm_noises_Z,replace_existing=True)
        self.radar.add_field_like('differential_reflectivity','ZDR_removed_noises', rm_noises_ZDR,replace_existing=True)
    def convert_grid(self):
        grid = pyart.map.grid_from_radars(
            self.radar,
            grid_shape=self.shape_grid, #Number of points in the grid (z, y, x)
            grid_limits=((0, 9000), (-200000, 200000), (-200000, 200000)), # min-max tuong duong z,y,x
            grid_origin = (self.lat_0, self.lon_0),
            fields=['Z_removed_noises'],
            roi_func='dist_beam',
            weighting_function='cressman')
        return grid
    def convert_lat_lon(self, grid):
        #conver distance to lat/lon
        for i in range(self.shape_grid[1]):
            geog = pyart.core.cartesian_to_geographic_aeqd(grid.x["data"][i],grid.y["data"][i],self.lon_0, self.lat_0, R=6370997.0)
            grid.x["data"][i] = geog[0]
            grid.y["data"][i] = geog[1]
        return grid
    def SHY95_algorithm(self,Zh):
        Zh.mask=0
        SHY95=np.zeros((self.shape_grid[1],self.shape_grid[2]))
        #Step 1: Intensity
        mask_st1=(Zh>=40)
        SHY95[mask_st1]=1
        #Step 2: Peakeness
        def MBG(xo,yo,n,r,Zh): #Mean Background Reflectivity
            y,x = np.ogrid[-xo:n-xo, -yo:n-yo]
            mask = x*x + y*y <= r*r
            Zh_none_O=Zh[xo,yo]
            Zbg_ = np.ma.masked_array(Zh[mask], Zh[mask] ==Zh_none_O) # remove value cycle centers
            Zbg = np.ma.masked_array(Zbg_, Zbg_ == 0) # remove 0 values
            return Zbg
        for xo in range (0,self.shape_grid[1],1):
            for yo in range (0,self.shape_grid[2],1):
                if SHY95[xo,yo]==0:
                    Zbg=MBG(xo,yo,self.shape_grid[1],5.5,Zh).mean()
                    deltaZh=Zh[xo,yo]-Zbg
                    if (Zbg < 42.43) and (deltaZh >= (10-Zbg**2/180)):
                        SHY95[xo,yo]=1
                    elif Zbg>=42.43 and deltaZh>=0:
                        SHY95[xo,yo]=1
        #Step3: Surrounding area
        def MBG_mask_r(xo,yo,n,r,): # mask array with r change
            y,x = np.ogrid[-xo:n-xo, -yo:n-yo]
            mask = x*x + y*y <= r*r
            return mask
        medium=[25,30,35,40]
        for xo in range (0,self.shape_grid[1],1):
            for yo in range (0,self.shape_grid[2],1):
                if SHY95[xo,yo]==1:
                    Zbg=MBG(xo,yo,self.shape_grid[1],5.5,Zh).mean()
                    if Zbg < medium[0]:
                        r=0.5 #1km
                    elif Zbg>=medium[0] and Zbg <medium[1]:
                        r=1   #2km
                    elif Zbg>=medium[1] and Zbg <medium[2]: 
                        r=1.5 #3km
                    elif Zbg>=medium[2] and Zbg <medium[3]:
                        r=2   #4km
                    elif Zbg>=medium[3]:
                        r=2.5 #5km
                    mask_st2=MBG_mask_r(xo,yo,self.shape_grid[1],r=r)
                    SHY95[mask_st2]=2
        #Step4: remaining ares as stratiform
        for q in range (0,self.shape_grid[1],1):
            for l in range (0,self.shape_grid[2],1):
                if Zh[q,l] > 0 and SHY95[q,l] !=2:
                    SHY95[q,l]=1
        return SHY95
    def plot_SHY95_products(self,lons,lats,SHY95,linkOUT):
        circle_points_50km= cargeo.Geodesic().circle(lon=103.51694002747536,lat=21.571390070021152, radius=50000, n_samples=180, endpoint=False)
        circle_points_100km= cargeo.Geodesic().circle(lon=103.51694002747536,lat=21.571390070021152, radius=100000, n_samples=180, endpoint=False)
        circle_points_200km= cargeo.Geodesic().circle(lon=103.51694002747536,lat=21.571390070021152, radius=200000, n_samples=180, endpoint=False)
        geom_50km=Polygon(circle_points_50km)
        geom_100km=Polygon(circle_points_100km)
        geom_200km=Polygon(circle_points_200km)
        #plot map with datas
        fig=plt.figure(figsize=(10,15),dpi=80)
        ax1=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
        ax1.add_feature(cfeat.LAND)
        ax1.add_feature(cfeat.OCEAN.with_scale('10m'))
        ax1.add_feature(cfeat.COASTLINE.with_scale('50m'))
        ax1.add_feature(cfeat.BORDERS.with_scale('50m'))
        state_provinces_lines=cfeat.NaturalEarthFeature(category='cultural',name='admin_1_states_provinces_lines',scale='10m',facecolor='none')
        ax1.add_feature(state_provinces_lines,edgecolor='gray')
        g1=ax1.gridlines(draw_labels=True,linestyle='--')
        g1.xlabels_top = False
        g1.ylabels_left = False
        g1.xlocator = mticker.FixedLocator([102.51694002747536, 103.51694002747536, 104.51694002747536, 105.51694002747536])
        g1.ylocator = mticker.FixedLocator([20.571390070021152,21.571390070021152,22.571390070021152])
        ax1.set_extent((grid.x['data'][0],grid.x['data'][-1],grid.y['data'][0],grid.y['data'][-1]),crs=ccrs.PlateCarree())
        ax1.add_geometries((geom_50km,), crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=1, linestyle='--')
        ax1.add_geometries((geom_100km,), crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=1, linestyle='--')
        ax1.add_geometries((geom_200km,), crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=1, linestyle='--')
        cMap = c.ListedColormap(['b','r'])
        z1_plot=ax1.pcolormesh(lons,lats,SHY95,shading='flat',cmap=cMap)
        ax1.scatter(x=103.51694002747536,y=21.571390070021152,transform=ccrs.PlateCarree(), color="black",s=50, alpha=1,marker='P')
        ax1.set(title="SHY95 "+ self.radar.time['units'][14:])
        plt.colorbar(z1_plot,ax=ax1,orientation='horizontal',ticks=[], label='stratiform                                 convection')
        linkPIC=linkOUT+radar.time['units'][14:-10]+radar.time['units'][-10:-7]+'-'+radar.time['units'][-6:-4]
        plt.savefig(linkPIC)
#Tạo giới hạn bán kính 200km    
y,x = np.ogrid[-100:201-100, -100:201-100]
mask = x*x + y*y > 100*100

In [5]:
Case='Embedded'
Case_dic={Case:[3]}
for r in Case_dic[Case]:
    #linkRAW='D:/data/Radars/'+Case+'/'+str(r)+'/'
    linkRAW='D:/radar datas 2022/5.2022/5/'
    #print(linkRAW)
    #os.mkdir('E:/newPic/Pic_SHY95/'+Case+'/'+str(r))
    #linkOUT='E:/newPic/Pic_SHY95/'+Case+'/'+str(r)+'/'
    linkOUT='E:/Pic_SHY95/2022/5.2022/'
    for file_name in os.listdir(linkRAW):
        if file_name[11:13]=="00" or file_name[11:13]=="20" or file_name[11:13]=="40":#or file_name[11:13]=="10" or file_name[11:13]=="30" or file_name[11:13]=="50":
            radar = pyart.io.read_sigmet(linkRAW+file_name)
            lat_0 = radar.latitude['data'][0]
            lon_0 = radar.longitude['data'][0]
            shape_grid = (7, 201,201)
            UF_File = UF(radar, shape_grid, lat_0, lon_0)
            UF_File.remove_noises()
            grid = UF_File.convert_grid()
            grid_lat_lon = UF_File.convert_lat_lon(grid)
            lon=grid_lat_lon.x['data']
            lat=grid_lat_lon.y['data']
            lons,lats=np.meshgrid(lon,lat)
            SHY95=np.ma.masked_array(ma.masked_equal(UF_File.SHY95_algorithm(grid.fields['Z_removed_noises']['data'][1,:,:]),0.0),mask)
            UF_File.plot_SHY95_products(lons,lats,SHY95,linkOUT)
            print(file_name)

PHA220515140004.RAWA53H
PHA220515142004.RAWA542
PHA220515144004.RAWA54L
PHA220515150004.RAWA555
PHA220520192004.RAWABAL
PHA220520194004.RAWABB5
PHA220520200004.RAWABBP
PHA220520202004.RAWABC8
PHA220520204004.RAWABCT
PHA220520210005.RAWABDB
PHA220520212004.RAWABDW
PHA220520214004.RAWABEE
PHA220520220004.RAWABEZ
PHA220520222004.RAWABFH
PHA220520224004.RAWABG2
PHA220520230004.RAWABGL
PHA220520232004.RAWABH5
PHA220520234004.RAWABHP
PHA220521000004.RAWABJ8
PHA220521102004.RAWAC25
PHA220521104004.RAWAC2P
PHA220521110004.RAWAC38
PHA220521112004.RAWAC3T
PHA220521114004.RAWAC4B
PHA220521120004.RAWAC4W
PHA220521122004.RAWAC5E
PHA220521124004.RAWAC5Z
PHA220521130004.RAWAC6H
PHA220521132004.RAWAC72
PHA220521134004.RAWAC7L
PHA220521140004.RAWAC85
PHA220521142004.RAWAC8P
PHA220521144004.RAWAC98
PHA220521150004.RAWAC9T
PHA220521152004.RAWACAB
PHA220521154004.RAWACAW
PHA220521160004.RAWACBE
PHA220521162004.RAWACBZ
PHA220521164005.RAWACCH
PHA220521170004.RAWACD2
PHA220521172004.RAWACDL
PHA220521174004.